Want a simple example of learning to reinforcement learn.

https://arxiv.org/abs/1611.05763


https://hackernoon.com/learning-policies-for-learning-policies-meta-reinforcement-learning-rl%C2%B2-in-tensorflow-b15b592a2ddf

In [2]:
import numpy as np
import tensorflow as tf
# import tensorflow_probability as tfp
# tfd = tfp.distributions

In [1]:
import trfl

/usr/pkg/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [ ]:
trfl.sarsa

In [49]:
!pip install --user tensorflow-probability --upgrade

  Using cached https://files.pythonhosted.org/packages/91/b6/49160bc0bd2855b58f79de575a8a07e56fbef48e89dbe8ca553f4e174f81/tensorflow_probability-0.4.0-py2.py3-none-any.whl
  Could not find a version that satisfies the requirement tensorflow>=1.10.0 (from tensorflow-probability) (from versions: )
No matching distribution found for tensorflow>=1.10.0 (from tensorflow-probability)


In [66]:
class Memory():
    def __init__(self):
        self.mem = [[]]
        self.counter = 0
    
    def append(self, x, a_old, r_old, done):
        self.mem[-1].append([x, a_old, r_old])
        if done:
            self.mem.append([])
            
    def get_batch(self, batch_size):
        idx = np.random.randint(0,len(self.mem), batch_size)
        return zip(*self.mem[idx])

In [72]:
M = Memory()
M.append(2,3,4, False)
M.append(12,1,3, True)

M.append(2,6,7, False)
M.append(3,2,4, False)
M.mem

[[[2, 3, 4], [12, 1, 3]], [[2, 6, 7], [3, 2, 4]]]

In [357]:
class A2C():
    def __init__(self, n_inputs, n_hidden, n_actions, time_steps=20, batch_size=50):
        self.time_steps = time_steps
        self.batch_size = batch_size
        self.n_inputs = n_inputs
        
        self.sess = tf.Session()
        
        self.a_old = None
        
        self.construct(n_inputs, n_hidden, n_actions)
        self.build_train_graph()
        self.build_worker_graph()
        
        self.memory = Memory()
        
    def construct(self, n_inputs, n_hidden, n_actions):
        self.encoder = tf.keras.Sequential([
            tf.keras.layers.Dense(n_hidden, input_shape=[None, n_inputs])
        ])
        
        self.rnn = tf.keras.layers.LSTMCell(n_hidden, input_shape=[None, n_inputs])
    
        self.value_net = tf.keras.Sequential([
            tf.keras.layers.Dense(10, input_shape=[None, n_hidden])
        ])
        
        self.policy_net = tf.keras.Sequential([
            tf.keras.layers.Dense(n_actions, input_shape=[None, n_hidden])
        ])

    def policy(self, h):
        z = self.policy_net(h)
        
        n = tf.shape(z)[-1]
        mu = z[:, :n//2]
        stddev = z[:, n//2:]
        
        dist = tfd.MultiVariateGaussian(shift=mu, scale=stddev)
        sample = dist.sample()
        return sample, dist
    
    def build_train_graph(self):
        self.inputs = tf.placeholder(shape=[self.time_steps, self.batch_size, self.n_inputs], dtype=tf.float32)
        self.actions = tf.placeholder(shape=[self.time_steps, self.batch_size, 1], dtype=tf.float32)
        self.rewards = tf.placeholder(shape=[self.time_steps, self.batch_size, 1], dtype=tf.float32)
        # wait a minute. how does this work if we only get reward at the end of an episode?
        state = tf.zeros(self.rnn.state_size)
        print(state)
        
        for i in range(self.time_steps):
            h = self.encoder(self.inputs[i])
            z = tf.concat([h, self.actions[i], self.rewards[i]], axis=-1)
            y, state = self.rnn(z, states=state)
            outputs.append(y)
            
        v = self.value_net(y)
#         dist = self.policy(y)
            
                
        # the 'targets' in this case.
        self.R = tf.reduce_sum(self.rewards, axis=0)  
        
        # make actions that result in high R more likely
        advantage = self.R - self.v  # [batch x 1] - [batch x 1] = [batch x 1]
        # v should be predicted per time step or once per episode?
#         self.policy_loss = tf.reduce_mean(self.dist.log_prob(self.actions) * advantage)
    
        self.value_loss = tf.losses.mean_squred_error(v, R)
        
#         variables = self.rnn.variables + self.encoder.variables
#         policy_gnvs = self.opt.compute_gradients(self.policy_loss, variables + self.policy_net.variables)
#         value_gnvs = self.opt.compute_gradients(self.value_loss, variables + self.value_net.variables)
#         self.train_op = self.opt.apply_gradients(policy_gnvs + value_gnvs)
    
    def build_worker_graph():
        self.input = tf.placeholder(shape=[1, self.n_inputs], dtype=tf.float32)
        self.action = tf.placeholder(shape=[1, 1], dtype=tf.float32)
        self.reward = tf.placeholder(shape=[1, 1], dtype=tf.float32)
        self.state = tf.placeholder([1, self.n_hidden])
        
        h = self.encoder(self.input)
        y, state = self.rnn(tf.concat([h, self.action, self.reward], axis=-1), self.state)

        v = self.value_net(y)
        dist = self.policy(y)
        self.sample = dist.sample()
    
    def __call__(self, x, r_old, done=False):
        if self.a_old is not None:
            self.memory.append(x, self.a_old, r_old, done)
        
        if self.memory.size > self.batch_size:
            self.run_train_step()
        
        feed = {self.input: x, self.action: a, self.reward: r, self.state: self.current_state}
        a, self.current_state = sess.run(self.sample, feed_dict=feed)

        self.a_old = a
        return a
    
    def run_train_step(self):
        """
        Take a training step on data from the memory store.
        """
        x, a, r = self.memory.get_batch(self.batch_size)
        feed = {self.inputs: x, self.actions: a, self.rewards: r}
        _, summ, L = self.sess.run([train_step, summaries], 
                      feed_dict=feed)
        return L

In [358]:
player = A2C(n_inputs=10, n_hidden=8, n_actions=2)

Tensor("zeros_8:0", shape=(8, 8), dtype=float32)
Tensor("concat_16:0", shape=(50, 10), dtype=float32)


ValueError: Shape must be rank 2 but is rank 1 for 'lstm_cell_3/MatMul_4' (op: 'MatMul') with input shapes: [8], [8,8].

TypeError: call() takes 2 positional arguments but 3 were given

In [156]:
player(10, 1)

0.22338637367119119

In [157]:
player.memory.mem

[[[10, 0.4737429771488225, 1],
  [10, 0.23962195034450706, 1],
  [10, 0.7315600680253059, 1],
  [10, 0.2723810689127708, 1],
  [10, 0.2454682824547253, 1],
  [10, 0.9309105824990254, 1],
  [10, 0.06696209301656353, 1],
  [10, 0.004830353714931124, 1],
  [10, 0.2290331402014153, 1],
  [10, 0.9662556100643971, 1],
  [10, 0.1311983168264812, 1],
  [10, 0.32105181802991456, 1]]]

In [292]:
class Env():
    def __init__(self, n_bandits):
        self.n_bandits = n_bandits
        self.means = np.random.standard_normal(n_bandits)
        self.vars = np.random.standard_normal(n_bandits)**2
        
        self.reset()
        self.maxsteps = 100
        
    def step(self, action):
        reward = self.means[action] + self.vars[action] * 0.1*np.random.standard_normal()
        
        if self.timestep > self.maxsteps:
            done = True
            self.reset()
        else:
            done = False
            self.timestep += 1
            
        return reward, done
    
    def reset(self):
        self.timestep = 0

In [277]:
bandit = Env(5)

In [291]:
bandit.step(2)

(0.5783304846031603, False)

In [237]:
player = A2C(n_inputs=10, n_hidden=8, n_actions=2)

In [ ]:
obs, r = env.reset()
while True:
    a = player(obs, r)
    obs, r = env.step(a)